# Markov model classifier

- [x] try removal of punktuation (how to achieve this with NLTK or spacy)
- [ ] try encoding an explicit ```<END-OF-LINE>``` marker as "terminal state"
- [ ] try training the word to index encoder by trained specifically for each poet
- [ ] try using back-of-words or tfidf to encode and classify the sentences

In [ ]:
%%bash
wget -nc -O data/edgar_allan_poe.txt https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
wget -nc -O data/robert_frost.txt https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

In [ ]:
import os
import re
from itertools import pairwise

import numpy as np
from nltk import word_tokenize as word_tokenize_nltk
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split

# import nltk  # noqa: E501
# nltk.download("punkt")  # noqa: E501

In [ ]:
def read_lines(fn):
    with open(fn, "r") as ifn:
        lines = ifn.readlines()

    # Clean up
    lines = [ln.rstrip("\n") for ln in lines]
    lines = [ln for ln in lines if ln != "\u2009"]
    lines = [ln for ln in lines if len(ln) > 0]

    return lines


def get_xy(fn, lab=None):
    x = read_lines(fn)
    if lab is None:
        lab = os.path.basename(fn).split(os.extsep)[0]
    y = len(x) * [lab]

    return x, y


def tokenize_document(doc, version):
    if version == "nltk":
        toks = word_tokenize_nltk(doc.lower())
    elif version == "re":
        toks = [tok.lower() for tok in re.findall(r"\w+", doc)]
    else:
        raise ValueError(f"Invalid tokeniser version: {version}")

    assert isinstance(toks, list)

    return toks


class Tok2IndexMapper(object):
    """A simple class to map tokens to indices and vice versa."""

    def __init__(self) -> None:
        self.missing_token_string = "<MISSING>"
        pass

    def fit(self, x):
        self._tok2idx = {}
        self._idx2toc = []
        idx = 0
        for doc in x:
            for token in doc:
                if token not in self._tok2idx:
                    self._tok2idx[token] = idx
                    self._idx2toc.append(token)
                    assert self._idx2toc[idx] == token
                    idx += 1

        # Add index for missing tokens
        assert self.missing_token_string not in self._tok2idx
        self._tok2idx[self.missing_token_string] = idx
        self._idx2toc.append(self.missing_token_string)
        assert self._idx2toc[idx] == self.missing_token_string

        return self

    def doc2idx(self, doc):
        return [self.tok2idx(tok) for tok in doc]

    def tok2idx(self, tok):
        try:
            idx = self._tok2idx[tok]
        except KeyError:
            idx = self._tok2idx[self.missing_token_string]

        return idx

    def idx2doc(self, idc):
        return [self.idx2toc(idx) for idx in idc]

    def idx2toc(self, idx):
        return self._idx2toc[idx]

    def __str__(self) -> str:
        return f"Vocab size: {len(self._tok2idx)}"

    def __len__(self) -> int:
        return len(self._tok2idx)


def estimate_lh_parameters(x, mapper, verbose=False):

    n = len(x)  # number of documents / sequences
    m = len(mapper)  # number of states

    # Count occurances
    ci_first = np.zeros((m,))
    ci = np.zeros((m,))
    for doc in x:
        ci_first[doc[0]] += 1  # first occurance
        for i in doc:
            ci[i] += 1  # any occurance

    if verbose:
        print(
            "top-10 words (first)\t:", mapper.idx2doc(np.argsort(ci_first)[::-1][:10])
        )
        print("top-10 words (all)\t:", mapper.idx2doc(np.argsort(ci)[::-1][:10]))

    # Count pairwise occurances
    cij = np.zeros((m, m))
    for doc in x:
        for i, j in pairwise(doc):
            cij[i, j] += 1

    if verbose:
        print(
            "top-10 word pairs\t:",
            [
                f"{mapper.idx2toc(i)} -> {mapper.idx2toc(j)}"
                for i, j in zip(
                    *np.unravel_index(np.argsort(cij.flatten())[::-1][:10], cij.shape)
                )
            ],
        )

    # Intitial state distribution
    pi = (ci_first + 1) / (n + m)
    assert np.allclose(np.sum(pi), 1)

    # State transition matrix
    a = cij + 1

    # Normalize
    a = a / a.sum(axis=1)[:, np.newaxis]
    assert np.allclose(np.sum(a, axis=1), 1)

    return pi, a


def estimate_prior_parameters(y):
    pi = np.bincount(y) / len(y)
    return pi


class Classifier:
    def __init__(self, logas, logpis, logpriors):
        self.logas = logas
        self.logpis = logpis
        self.logpriors = logpriors
        self.K = len(logpriors)  # number of classes

    def _compute_log_likelihood(self, input_, class_):
        loga = self.logas[class_]
        logpi = self.logpis[class_]

        last_idx = None
        logprob = 0
        for idx in input_:
            if last_idx is None:
                # it's the first token
                logprob += logpi[idx]
            else:
                logprob += loga[last_idx, idx]

            # update last_idx
            last_idx = idx

        return logprob

    def predict(self, inputs):
        predictions = np.zeros(len(inputs))
        for i, input_ in enumerate(inputs):
            posteriors = [
                self._compute_log_likelihood(input_, c) + self.logpriors[c]
                for c in range(self.K)
            ]
            pred = np.argmax(posteriors)
            predictions[i] = pred
        return predictions

## Load the data

In [ ]:
x_poe, y_poe = get_xy("data/edgar_allan_poe.txt", 0)
x_frost, y_frost = get_xy("data/robert_frost.txt", 1)

print(x_poe[:3])
print(y_poe[:3])

# Combine poets
x = x_poe + x_frost
y = y_poe + y_frost

## Prepocess the data

1. tokenize
2. split to train test
3. train the token to index mapper

In [ ]:
def prepocess_data(x, y, tokeniser_version, separate_mapper):
    assert tokeniser_version in ["nltk", "re"]
    assert separate_mapper in [True, False]

    # Tokenize the documents
    x = [tokenize_document(doc, tokeniser_version) for doc in x]

    # Split data into training and test sets
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.25, random_state=42
    )

    # Train the mapper on the training data
    labs = sorted(set(y_train))
    if separate_mapper:
        mappers = [
            Tok2IndexMapper().fit(
                [doc for doc, labi in zip(x_train, y_train) if labi == lab]
            )
            for lab in labs
        ]
    else:
        mappers = [Tok2IndexMapper().fit(x_train)] * len(labs)

    # Convert the data to indices
    x_train = [mappers[lab].doc2idx(doc) for doc, lab in zip(x_train, y_train)]
    x_test = [mappers[lab].doc2idx(doc) for doc, lab in zip(x_test, y_test)]

    return x_train, x_test, y_train, y_test, mappers

## Train and evalute the models

### 1. Preprocess the data
### 2. Estimate the parameters of the Markov model

**Likelihood**
- Transition matrix $\mathbf{A}$
- Intitial state propabilities $\pi$

**Priors**
- Priors $\mathbf{p}$

### 3. Predict and evaluate

## Version 1: NLTK tokenizer, single mapper

In [ ]:
# Preprocess the data
x_train, x_test, y_train, y_test, mappers = prepocess_data(x, y, "nltk", False)

# Estimate the parameters
pi_0, A_0 = estimate_lh_parameters(
    [x for x, y in zip(x_train, y_train) if y == 0], mappers[0], verbose=False
)
pi_1, A_1 = estimate_lh_parameters(
    [x for x, y in zip(x_train, y_train) if y == 1], mappers[1], verbose=False
)

priors = estimate_prior_parameters(y_train)

# build a classifier
clf = Classifier(
    [np.log(A_0), np.log(A_1)], [np.log(pi_0), np.log(pi_1)], np.log(priors).tolist()
)

# Evaluate
y_hat_train = clf.predict(x_train)
print(f"Train acc: {np.mean(y_hat_train == y_train)}")
print(f"Train f1: {f1_score(y_train, y_hat_train)}")
print(confusion_matrix(y_train, y_hat_train))

y_hat_test = clf.predict(x_test)
print(f"Test acc: {np.mean(y_hat_test == y_test)}")
print(f"Test f1: {f1_score(y_test, y_hat_test)}")
print(confusion_matrix(y_test, y_hat_test))

## Version 2: RE tokenizer, single mapper

In [ ]:
# Preprocess the data
x_train, x_test, y_train, y_test, mappers = prepocess_data(x, y, "re", False)

# Estimate the parameters
pi_0, A_0 = estimate_lh_parameters(
    [x for x, y in zip(x_train, y_train) if y == 0], mappers[0], verbose=False
)
pi_1, A_1 = estimate_lh_parameters(
    [x for x, y in zip(x_train, y_train) if y == 1], mappers[1], verbose=False
)

priors = estimate_prior_parameters(y_train)

# build a classifier
clf = Classifier(
    [np.log(A_0), np.log(A_1)], [np.log(pi_0), np.log(pi_1)], np.log(priors).tolist()
)

# Evaluate
y_hat_train = clf.predict(x_train)
print(f"Train acc: {np.mean(y_hat_train == y_train)}")
print(f"Train f1: {f1_score(y_train, y_hat_train)}")
print(confusion_matrix(y_train, y_hat_train))

y_hat_test = clf.predict(x_test)
print(f"Test acc: {np.mean(y_hat_test == y_test)}")
print(f"Test f1: {f1_score(y_test, y_hat_test)}")
print(confusion_matrix(y_test, y_hat_test))

## Version 3: NLTK tokenizer, separate mapper

In [ ]:
# Preprocess the data
x_train, x_test, y_train, y_test, mappers = prepocess_data(x, y, "nltk", True)

# Estimate the parameters
pi_0, A_0 = estimate_lh_parameters(
    [x for x, y in zip(x_train, y_train) if y == 0], mappers[0], verbose=False
)
pi_1, A_1 = estimate_lh_parameters(
    [x for x, y in zip(x_train, y_train) if y == 1], mappers[1], verbose=False
)
print(pi_0.shape, A_0.shape)
print(pi_1.shape, A_1.shape)
priors = estimate_prior_parameters(y_train)

# build a classifier
clf = Classifier(
    [np.log(A_0), np.log(A_1)], [np.log(pi_0), np.log(pi_1)], np.log(priors).tolist()
)

# Evaluate
# TODO: When using separate mappers the indices are not consistent. We need to re-apply
# each mapper to the data which should be predicted. This is not implemented yet.
y_hat_train = clf.predict(x_train)
print(f"Train acc: {np.mean(y_hat_train == y_train)}")
print(f"Train f1: {f1_score(y_train, y_hat_train)}")
print(confusion_matrix(y_train, y_hat_train))

y_hat_test = clf.predict(x_test)
print(f"Test acc: {np.mean(y_hat_test == y_test)}")
print(f"Test f1: {f1_score(y_test, y_hat_test)}")
print(confusion_matrix(y_test, y_hat_test))